# 🧩 Week 7-8 · Notebook 03 · Dynamic Prompts with Templates & Chains

**Module:** LangChain, Agents, & Advanced RAG  
**Project:** Manufacturing Copilot - Building Reusable and Auditable Prompts

---

### From Hardcoded to Dynamic: The Power of Prompt Templates

In our previous notebooks, we created prompts by manually formatting strings. This works for simple examples, but it's not a scalable or maintainable approach for production applications. What happens when you need to change the instructions? Or use the same prompt structure with different user inputs?

This is where **Prompt Templates** come in.

A `PromptTemplate` is a reusable and parameterizable object that defines the structure of your prompt. It separates the fixed instructions from the dynamic, user-provided data. This allows you to:

-   **Standardize:** Ensure every prompt sent to the LLM follows a consistent, optimized format.
-   **Reuse:** Use the same prompt structure across different parts of your application.
-   **Maintain:** Update the core prompt logic in one central place without changing your application code.

In this notebook, we will learn how to create dynamic, production-ready prompts using `ChatPromptTemplate` and compose them into a complete chain. We'll also implement a crucial best practice: an automated validation harness to ensure our prompts and the LLM's responses meet our quality and safety standards.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to build robust, maintainable, and auditable prompt engineering workflows. You will learn to:

1.  **Build Parameterized `ChatPromptTemplate`s:** Create dynamic prompts that can be populated with variables at runtime.
2.  **Inject Configuration Data:** Design prompts that can be customized with external configuration, such as plant-specific safety disclaimers or compliance standards.
3.  **Compose a Complete LCEL Chain:** Combine a `ChatPromptTemplate`, an `LLM`, and an `OutputParser` into a seamless, executable pipeline.
4.  **Create an Automated Validation Harness:** Write a function to programmatically check the LLM's output for critical requirements, such as the presence of a safety warning or a citation.

## ⚙️ Step 1: Environment Setup

Let's start by installing the necessary libraries and setting up our OpenAI API key.

In [ ]:
# Install necessary packages
!pip install -q langchain langchain-openai python-dotenv

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
    print("API Key not found. Please set it as an environment variable or replace 'YOUR_API_KEY'.")
else:
    print("OpenAI API Key loaded successfully.")

## 🏭 Step 2: Creating a Dynamic, Configurable Prompt

Imagine our Manufacturing Copilot needs to be deployed across multiple plants. Each plant might have slightly different compliance rules or safety warnings. Hardcoding these into the prompt is not feasible.

We need a template that can be dynamically configured.

Let's create a `ChatPromptTemplate` that takes not only the user's question and the retrieved context, but also plant-specific configuration data. This makes our prompt a reusable asset that can be adapted to different environments without changing its core logic.

### The Scenario
Corporate Safety has issued a new, mandatory safety disclaimer for all maintenance-related AI assistance. We need to ensure this new disclaimer is included in all responses, and we need to do it by changing a configuration file, not by editing every prompt in our codebase.

We will simulate this by creating:
1.  A `plant_config` dictionary containing the plant-specific data.
2.  A `ChatPromptTemplate` with placeholders for this configuration.
3.  A complete chain that uses this template.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# --- 1. Define Plant-Specific Configuration ---
# In a real application, this would be loaded from a config file (e.g., YAML, JSON) or a database.
plant_config = {
    "plant_id": "PUNE-01",
    "language": "en",
    "compliance_grade": "ISO 9001",
    "safety_disclaimer": "CRITICAL SAFETY WARNING: Always follow official Lockout/Tagout (LOTO) procedures as per SOP-900 before beginning any maintenance. Ensure a zero-energy state is confirmed."
}

# --- 2. Create a Parameterized Prompt Template ---
# This template uses f-string style placeholders `{variable_name}` for all dynamic inputs.
template_string = """
You are a Manufacturing Copilot for Plant {plant_id}.
Your response must adhere to {compliance_grade} standards and be in {language}.

**Mandatory Safety Protocol:**
{safety_disclaimer}

---
**Retrieved Context from SOP:**
{sop_snippet}

---
**Technician's Question:**
{question}

---
Provide a concise, factual answer based ONLY on the provided SOP context.
"""

prompt_template = ChatPromptTemplate.from_template(template_string)


# --- 3. Build the LCEL Chain ---
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
chain = prompt_template | llm | StrOutputParser()


# --- 4. Invoke the Chain with All Required Inputs ---
# The `invoke` method requires a dictionary containing values for ALL variables in the template.
sop_context = "SOP-112, Section 4.1 states: 'The primary hydraulic pump requires a full filter replacement every 500 operating hours or 3 months, whichever comes first.'"
question = "How often do I need to replace the hydraulic pump filter?"

answer = chain.invoke({
    # Configuration variables
    "plant_id": plant_config["plant_id"],
    "language": plant_config["language"],
    "compliance_grade": plant_config["compliance_grade"],
    "safety_disclaimer": plant_config["safety_disclaimer"],
    
    # Runtime variables
    "sop_snippet": sop_context,
    "question": question
})

print("--- Generated Response ---")
print(answer)

## 🧪 Step 3: Building a Validation Harness

Generating a response is only half the battle. In a production environment, we need to be sure the response meets our quality and safety standards. Did the LLM follow our instructions? Did it include the mandatory safety warning?

To automate this, we can create a **validation harness**. This is a function that programmatically checks the LLM's output against a set of rules. This is a form of "testing" for our prompts and models.

Our harness will check for three things:
1.  **Disclaimer Presence:** Did the response include the exact safety disclaimer we provided?
2.  **SOP Citation:** Did the response reference the SOP document (e.g., "SOP-112")?
3.  **Conciseness:** Is the response reasonably short and to the point?

This automated check is crucial for CI/CD (Continuous Integration/Continuous Deployment) workflows. Before deploying a new prompt or model, you can run it through the validation harness to prevent regressions and ensure compliance.

import re

def validate_response(response_text: str, config: dict) -> dict:
    """
    Validates the LLM's response against a set of predefined rules.

    Args:
        response_text: The text generated by the LLM.
        config: The plant configuration dictionary containing the rules.

    Returns:
        A dictionary summarizing the validation results.
    """
    disclaimer = config.get("safety_disclaimer", "")
    
    validation_results = {
        "has_disclaimer": disclaimer in response_text,
        "has_sop_citation": bool(re.search(r"SOP-\d+", response_text)),
        "is_concise": len(response_text.split()) < 75  # Example constraint: response should be under 75 words.
    }
    return validation_results

# --- Run Validation on Our Previous Answer ---
validation = validate_response(answer, plant_config)

print("--- Validation Harness Results ---")
print(f"Response Validated: '{answer[:50]}...'")
print(json.dumps(validation, indent=2))

# --- Automated Assertions for CI/CD ---
# In a real test suite, you would use assertions to automatically pass or fail a build.
try:
    assert validation['has_disclaimer'], 'CRITICAL: Safety disclaimer is missing from the response!'
    assert validation['has_sop_citation'], 'CRITICAL: SOP citation is missing from the response!'
    print("\n✅ All critical validation checks passed.")
except AssertionError as e:
    print(f"\n❌ VALIDATION FAILED: {e}")

## ✅ Congratulations and Next Steps!

You have now learned how to create dynamic, reusable, and auditable AI workflows using LangChain's `ChatPromptTemplate`.

You've seen how to separate configuration from logic, allowing you to adapt your prompts to different environments without changing your code. Most importantly, you've implemented a validation harness—a critical tool for ensuring the quality, safety, and reliability of your AI's responses in a production setting.

### Key Takeaways:
-   **Templates are for Maintainability:** `ChatPromptTemplate` is the key to building scalable applications. It allows you to manage your prompt logic centrally.
-   **Configuration Drives Flexibility:** By injecting configuration into your templates, you can create a single, reusable prompt that serves multiple purposes.
-   **Validation Ensures Reliability:** Never trust, always verify. An automated validation harness is your first line of defense against prompt regressions and model misbehavior.

In the next notebook, **`04_runnable_sequences.ipynb`**, we will dive deeper into the LangChain Expression Language (LCEL) and explore more advanced ways to construct complex, multi-step chains. Let's continue our journey to mastering LangChain!